In [ ]:
def functionality_2 (graph,metric, node, n_heroes = None):
    global whole_output

    if n_heroes is not None:
        n = findNHeroes(df_edges,n_heroes)

    if metric == 'Betweenness':
        betweenness = nx.betweenness_centrality(graph,k=10, seed=1234, weight='weight')

        output = pd.DataFrame.from_dict(betweenness, orient='index',
                               columns=['Betweenness']).sort_values(by='Betweenness', ascending=False)
        try:
            whole_output = output.loc[n,:].sort_values(by='Betweenness',ascending=False)
            return output.loc[node,:], whole_output
        except:
            return output.loc[node,:], output
    if metric == 'ClosenessCentrality':
        closeness = nx.closeness_centrality(graph,distance='weight')
        output = pd.DataFrame.from_dict(closeness, orient='index',
                               columns=['Closeness']).sort_values(by='Closeness', ascending=False)
        try:
            whole_output = output.loc[n,:].sort_values(by='Closeness',ascending=False)
            return output.loc[node,:], whole_output
        except:
            return output.loc[node,:], output

    if metric == 'PageRank':
        pagerank = nx.pagerank(graph, weight='weight')
        output = pd.DataFrame.from_dict(pagerank, orient='index',
                               columns=['PageRank']).sort_values(by='PageRank', ascending=False)
        try:
            whole_output = output.loc[n,:].sort_values(by='PageRank',ascending=False)
            return output.loc[node,:], whole_output
        except:
            return output.loc[node,:], output

    if metric == 'DegreeCentrality':
        degree = nx.degree_centrality(graph)
        output = pd.DataFrame.from_dict(degree, orient='index',
                               columns=['DegreeCentrality']).sort_values(by='DegreeCentrality', ascending=False)
        try:
            whole_output = output.loc[n,:].sort_values(by='DegreeCentrality',ascending=False)
            return output.loc[node,:], whole_output
        except:
            return output.loc[node,:], output

In [ ]:
def visualization_2(graph, metric, node, n_heroes=None):

    given_node, out = functionality_2(graph,metric,node,n_heroes)

    df = pd.DataFrame(given_node)

    df['Average'+ f' {metric} '+'measure'] = float(out.values.mean())

    return df

In [ ]:
def functionality_4(graph,source,target,top_n = None):
    if top_n is not None:
        n = set(findNHeroes(e, top_n)) ## denoting the top N heroes that our algorithm should be considered

    G = graph.copy()
    F = nx.subgraph(G,n) ## creating the subgraph with the top N heroes
    H = F.copy()

    cutset = nx.minimum_edge_cut(H,s=source,t=target) ## the set of the minimum number of edges to disconnect the source from the target
    H.remove_edges_from(cutset) ## remove the cutset from the original graph



    sub_source_edges = set(bfs_unweighted(H,source = source))
    sub_target_edges = set(bfs_unweighted(H,source = target))

    sub_source, sub_target = nx.Graph(), nx.Graph()
    sub_source.add_edges_from(sub_source_edges)
    sub_target.add_edges_from(sub_target_edges)

    return (cutset),F, H, sub_source, sub_target

In [ ]:
def visualization_4(graph,source,target,top_n = None):
    cutset,G_connected, G_disconnected, sub_source, sub_target = functionality_4(graph,source,target,top_n)
    cut = nx.Graph()
    cut.add_edges_from(cutset)
    H = nx.difference(G_connected,cut)

    print(f'The minimum number of the links that should be disconnected in order to get two disconnected sub-graphs is {len(cutset)}')

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 15))
    pos = nx.circular_layout(G_connected)
    nodes = ['red' if (node == source or node == target) else 'blue' for node in G_connected.nodes()]
    nx.draw(G_connected,pos,with_labels=True,ax=ax1,node_color = nodes)
    nx.draw(H,pos,with_labels=True,ax=ax2, node_color = nodes)
    ax1.set_title('Plot of the original graph', fontweight = 'bold', size = 15)
    ax2.set_title('Plot of the graph after removing the links between the two nodes identified', fontweight = 'bold', size = 15)
    plt.show()

In [ ]:
visualization_4(G,'IRON MAN/TONY STARK','HULK/DR. ROBERT BRUC',top_n=15)

In [ ]:
def bfs_weighted(graph,source):

    Q = Queue()
    graph_dict = nx.to_dict_of_dicts(graph)
    visited = {key: False for key in graph_dict.keys()}

    Q.put((source,source,0))
    visited[source] = True
    bfs_edges = list()

    while Q.empty()==False:
            s,path,weight = Q.get(0)

            if isinstance(weight,dict):
                bfs_edges.append((path,s,weight['weight']))

            for neighbor,weight in graph_dict[s].items():
                if visited[neighbor] == False:
                    Q.put((neighbor,s,weight))
                    visited[neighbor] = True


    return bfs_edges

In [ ]:
def bfs_unweighted(graph,source):
    Q = Queue()
    graph_dict = nx.to_dict_of_dicts(graph)
    visited = {key: False for key in graph_dict.keys()}

    Q.put((source,source))
    visited[source] = True
    bfs_edges = list()
    try:
        while Q:
            s,path = Q.get(0)


            if s != source:
                bfs_edges.append((path,s))

            for neighbor in graph_dict[s].keys():
                if visited[neighbor] == False:
                    Q.put((neighbor,s))
                    visited[neighbor] = True

    except:
        pass
    return bfs_edges
